In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [16]:
data = pd.read_csv("cleaned_merged_seasons.csv")

<ipython-input-16-ce1e41c9b2f4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_merged_seasons.csv")


In [17]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normaliser les caractéristiques
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
scaler = MinMaxScaler()
data[features_to_normalize] = scaler.fit_transform(data[features_to_normalize])

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

In [18]:
print(player_names)

0                  Aaron Connolly
1                 Aaron Cresswell
2                      Aaron Mooy
3                  Aaron Ramsdale
4              Abdoulaye Doucouré
                   ...           
76312                Oliver Skipp
76313              Ryan Sessegnon
76314                Ashley Young
76315    Jeremy Sarmiento Morante
76316              Philip Billing
Name: name, Length: 76317, dtype: object


In [19]:
# Séparation des données en entrée (X) et sortie (y)
X = data_encoded.drop('total_points', axis=1)
y = data_encoded['total_points']

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(
    X, y, player_names, test_size=0.2, random_state=42
)

# Redimensionnement des données pour être utilisées dans un modèle LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))


In [20]:
# Création du modèle LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [21]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
1908/1908 [==============================] - 21s 9ms/step - loss: 5.4913 - val_loss: 5.4969
Epoch 2/50
1908/1908 [==============================] - 14s 7ms/step - loss: 5.2206 - val_loss: 5.1681
Epoch 3/50
1908/1908 [==============================] - 14s 7ms/step - loss: 5.0977 - val_loss: 5.1236
Epoch 4/50
1908/1908 [==============================] - 13s 7ms/step - loss: 5.1497 - val_loss: 5.1482
Epoch 5/50
1908/1908 [==============================] - 13s 7ms/step - loss: 5.0556 - val_loss: 5.0471
Epoch 6/50
1908/1908 [==============================] - 13s 7ms/step - loss: 5.1770 - val_loss: 5.4861
Epoch 7/50
1908/1908 [==============================] - 15s 8ms/step - loss: 5.2872 - val_loss: 5.2086
Epoch 8/50
1908/1908 [==============================] - 14s 7ms/step - loss: 5.3944 - val_loss: 5.5124
Epoch 9/50
1908/1908 [==============================] - 14s 7ms/step - loss: 5.4568 - val_loss: 5.1292
Epoch 10/50
1908/1908 [==============================] - 14s 7ms/step - l

In [24]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

477/477 [==============================] - 1s 2ms/step


In [26]:
player_names_test = player_names_test.reset_index(drop=True)

# Assurez-vous que y_pred est un tableau 1D
y_pred_1d = y_pred.flatten()

# Créez le DataFrame avec les colonnes unidimensionnelles
results_df = pd.DataFrame({'Name': player_names_test, 'Predicted_Total_Points': y_pred_1d})

print(results_df)

                    Name  Predicted_Total_Points
0           Daniel James                2.064900
1           Nicolas Pépé                2.064900
2             Sam Waller                0.466208
3             Bernd Leno                2.064900
4       Ezri Konsa Ngoyo                2.064900
...                  ...                     ...
15259  Aaron Wan-Bissaka                2.064900
15260       Divock Origi                2.064900
15261   Joseph Whitworth                0.466208
15262       Alex Mighten                0.466208
15263  Christian Pulisic                2.064900

[15264 rows x 2 columns]


In [23]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

477/477 [==============================] - 1s 2ms/step
Mean Squared Error: 5.23560792457459
Mean Absolute Error: 1.3619685606011804
R^2 Score: 0.11310214620377357
